# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 24 2022 12:47PM,249261,10,8686708,"Snap Medical Industries, LLC",Released
1,Oct 24 2022 12:47PM,249261,10,8686709,"Snap Medical Industries, LLC",Released
2,Oct 24 2022 12:47PM,249261,10,8686712,"Snap Medical Industries, LLC",Released
3,Oct 24 2022 12:47PM,249261,10,8686713,"Snap Medical Industries, LLC",Released
4,Oct 24 2022 12:47PM,249261,10,8686715,"Snap Medical Industries, LLC",Released
5,Oct 24 2022 12:47PM,249261,10,8686720,"Snap Medical Industries, LLC",Released
6,Oct 24 2022 12:46PM,249260,10,8688484,Yusen – 3D Matrix,Released
7,Oct 24 2022 12:34PM,249259,19,ACG-2102491653,ACG North America LLC,Released
8,Oct 24 2022 12:14PM,249256,19,ADV80006922,"AdvaGen Pharma, Ltd",Released
9,Oct 24 2022 12:06PM,249253,19,ADV80006905,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,249253,Released,11
15,249256,Released,1
16,249259,Released,1
17,249260,Released,1
18,249261,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249253,NaN,NaN,11.0
249256,NaN,NaN,1.0
249259,NaN,NaN,1.0
249260,NaN,NaN,1.0
249261,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249218,0.0,0.0,1.0
249223,2.0,4.0,1.0
249225,0.0,0.0,53.0
249228,0.0,0.0,60.0
249229,0.0,0.0,59.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249218,0,0,1
249223,2,4,1
249225,0,0,53
249228,0,0,60
249229,0,0,59


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249218,0,0,1
1,249223,2,4,1
2,249225,0,0,53
3,249228,0,0,60
4,249229,0,0,59


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249218,,,1
1,249223,2,4,1
2,249225,,,53
3,249228,,,60
4,249229,,,59


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 24 2022 12:47PM,249261,10,"Snap Medical Industries, LLC"
6,Oct 24 2022 12:46PM,249260,10,Yusen – 3D Matrix
7,Oct 24 2022 12:34PM,249259,19,ACG North America LLC
8,Oct 24 2022 12:14PM,249256,19,"AdvaGen Pharma, Ltd"
9,Oct 24 2022 12:06PM,249253,19,"AdvaGen Pharma, Ltd"
20,Oct 24 2022 11:59AM,249252,18,Innova Softgel LLC
21,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC"
48,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc."
49,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation
51,Oct 24 2022 10:06AM,249234,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 24 2022 12:47PM,249261,10,"Snap Medical Industries, LLC",,,6
1,Oct 24 2022 12:46PM,249260,10,Yusen – 3D Matrix,,,1
2,Oct 24 2022 12:34PM,249259,19,ACG North America LLC,,,1
3,Oct 24 2022 12:14PM,249256,19,"AdvaGen Pharma, Ltd",,,1
4,Oct 24 2022 12:06PM,249253,19,"AdvaGen Pharma, Ltd",,,11
5,Oct 24 2022 11:59AM,249252,18,Innova Softgel LLC,,,1
6,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",,11,16
7,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc.",,,1
8,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation,,,2
9,Oct 24 2022 10:06AM,249234,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 12:47PM,249261,10,"Snap Medical Industries, LLC",6,,
1,Oct 24 2022 12:46PM,249260,10,Yusen – 3D Matrix,1,,
2,Oct 24 2022 12:34PM,249259,19,ACG North America LLC,1,,
3,Oct 24 2022 12:14PM,249256,19,"AdvaGen Pharma, Ltd",1,,
4,Oct 24 2022 12:06PM,249253,19,"AdvaGen Pharma, Ltd",11,,
5,Oct 24 2022 11:59AM,249252,18,Innova Softgel LLC,1,,
6,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",16,11,
7,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc.",1,,
8,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation,2,,
9,Oct 24 2022 10:06AM,249234,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 12:47PM,249261,10,"Snap Medical Industries, LLC",6,,
1,Oct 24 2022 12:46PM,249260,10,Yusen – 3D Matrix,1,,
2,Oct 24 2022 12:34PM,249259,19,ACG North America LLC,1,,
3,Oct 24 2022 12:14PM,249256,19,"AdvaGen Pharma, Ltd",1,,
4,Oct 24 2022 12:06PM,249253,19,"AdvaGen Pharma, Ltd",11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 12:47PM,249261,10,"Snap Medical Industries, LLC",6,NaN,NaN
1,Oct 24 2022 12:46PM,249260,10,Yusen – 3D Matrix,1,NaN,NaN
2,Oct 24 2022 12:34PM,249259,19,ACG North America LLC,1,NaN,NaN
3,Oct 24 2022 12:14PM,249256,19,"AdvaGen Pharma, Ltd",1,NaN,NaN
4,Oct 24 2022 12:06PM,249253,19,"AdvaGen Pharma, Ltd",11,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 12:47PM,249261,10,"Snap Medical Industries, LLC",6,0.0,0.0
1,Oct 24 2022 12:46PM,249260,10,Yusen – 3D Matrix,1,0.0,0.0
2,Oct 24 2022 12:34PM,249259,19,ACG North America LLC,1,0.0,0.0
3,Oct 24 2022 12:14PM,249256,19,"AdvaGen Pharma, Ltd",1,0.0,0.0
4,Oct 24 2022 12:06PM,249253,19,"AdvaGen Pharma, Ltd",11,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1993880,183,4.0,2.0
15,249241,16,11.0,0.0
18,249252,1,0.0,0.0
19,997000,14,0.0,0.0
20,249242,1,0.0,0.0
21,249234,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1993880,183,4.0,2.0
1,15,249241,16,11.0,0.0
2,18,249252,1,0.0,0.0
3,19,997000,14,0.0,0.0
4,20,249242,1,0.0,0.0
5,21,249234,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,183,4.0,2.0
1,15,16,11.0,0.0
2,18,1,0.0,0.0
3,19,14,0.0,0.0
4,20,1,0.0,0.0
5,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,183.0
1,15,Released,16.0
2,18,Released,1.0
3,19,Released,14.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,4.0,11.0,0.0,0.0,0.0,0.0
Released,183.0,16.0,1.0,14.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,11.0,0.0,0.0,0.0,0.0
2,Released,183.0,16.0,1.0,14.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,11.0,0.0,0.0,0.0,0.0
2,Released,183.0,16.0,1.0,14.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()